In [1]:
import os

NOAA_TOKEN = os.getenv("NOAA_TOKEN")

if not NOAA_TOKEN:
    raise ValueError("Missing NOAA_TOKEN environment variable. You must export it in Terminal BEFORE starting JupyterLab.")

print("NOAA_TOKEN loaded successfully.")

NOAA_TOKEN loaded successfully.


In [2]:
# ===============================
# 1. IMPORT LIBRARIES
# ===============================

import pandas as pd
import numpy as np
import os
import zipfile

print("Libraries imported successfully.")

Libraries imported successfully.


In [3]:
import os
import requests
import pandas as pd

# ==================================================
# NOAA: Download DAILY weather data for LaGuardia (LGA) in 2022
# Dataset: GHCND (Daily Summaries)
# Station: USW00014732 (New York LaGuardia Airport)
# ==================================================

NOAA_TOKEN = os.getenv("NOAA_TOKEN")
if not NOAA_TOKEN:
    raise ValueError("NOAA_TOKEN is missing. Close Jupyter, export NOAA_TOKEN in Terminal, then reopen Jupyter.")

STATION_ID = "GHCND:USW00014732"
START_DATE = "2022-01-01"
END_DATE = "2022-12-31"
BASE_URL = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"

def fetch_noaa_daily(station_id, start_date, end_date, datatypes, token):
    headers = {"token": token}
    all_results = []
    limit = 1000
    offset = 1

    while True:
        params = {
            "datasetid": "GHCND",
            "stationid": station_id,
            "startdate": start_date,
            "enddate": end_date,
            "units": "metric",
            "limit": limit,
            "offset": offset,
            # IMPORTANT: NOAA expects repeated datatypeid params. requests handles this when value is a list.
            "datatypeid": datatypes,
        }

        r = requests.get(BASE_URL, headers=headers, params=params)

        if r.status_code != 200:
            # Show NOAA's error message (very helpful)
            raise RuntimeError(f"NOAA request failed: {r.status_code}\nURL: {r.url}\nResponse: {r.text}")

        data = r.json()
        results = data.get("results", [])
        all_results.extend(results)

        if len(results) < limit:
            break

        offset += limit

    return pd.DataFrame(all_results)

datatypes = ["TAVG", "TMIN", "TMAX", "PRCP"]

noaa_raw = fetch_noaa_daily(
    station_id=STATION_ID,
    start_date=START_DATE,
    end_date=END_DATE,
    datatypes=datatypes,
    token=NOAA_TOKEN
)

print("Rows downloaded:", len(noaa_raw))
print("Columns:", list(noaa_raw.columns))
noaa_raw.head()

Rows downloaded: 1460
Columns: ['date', 'datatype', 'station', 'attributes', 'value']


,date,datatype,station,attributes,value
0,2022-01-01T00:00:00,PRCP,GHCND:USW00014732,",,W,2400",19.3
1,2022-01-01T00:00:00,TAVG,GHCND:USW00014732,"H,,S,",11.6
2,2022-01-01T00:00:00,TMAX,GHCND:USW00014732,",,W,2400",13.9
3,2022-01-01T00:00:00,TMIN,GHCND:USW00014732,",,W,2400",10.0
4,2022-01-02T00:00:00,PRCP,GHCND:USW00014732,",,W,2400",1.0


In [4]:
# ==================================================
# CLEAN & RESHAPE NOAA DATA
# ==================================================

# Convert date column to datetime
noaa_raw["date"] = pd.to_datetime(noaa_raw["date"])

# Pivot from long to wide format
noaa_df = (
    noaa_raw
    .pivot(index="date", columns="datatype", values="value")
    .reset_index()
)

# Sort by date
noaa_df = noaa_df.sort_values("date")

print("Weather dataset shape:", noaa_df.shape)
noaa_df.head()

Weather dataset shape: (365, 5)


datatype,date,PRCP,TAVG,TMAX,TMIN
0,2022-01-01,19.3,11.6,13.9,10.0
1,2022-01-02,1.0,11.4,15.6,3.9
2,2022-01-03,0.0,1.4,3.9,-4.3
3,2022-01-04,0.0,-2.7,2.2,-6.0
4,2022-01-05,6.1,3.2,8.9,0.0


In [5]:
# ==================================================
# EXPORT CLEAN WEATHER DATA TO CSV
# ==================================================

noaa_df.to_csv("weather_2022_laguardia.csv", index=False)

print("Weather CSV exported successfully.")

Weather CSV exported successfully.


In [8]:
# ==================================================
# 5. AGGREGATE CITIBIKE DATA TO DAILY LEVEL
# ==================================================

import pandas as pd

# Convert started_at to datetime
citibike_df["started_at"] = pd.to_datetime(citibike_df["started_at"], errors="coerce")

# Drop rows where datetime failed
citibike_df = citibike_df.dropna(subset=["started_at"])

# Keep only 2022 rows (important if files contain spillover dates)
citibike_df = citibike_df[
    (citibike_df["started_at"] >= "2022-01-01") &
    (citibike_df["started_at"] <= "2022-12-31")
]

# Create date column
citibike_df["date"] = citibike_df["started_at"].dt.date

# Count number of rides per day
daily_rides = (
    citibike_df
    .groupby("date")
    .size()
    .reset_index(name="daily_ride_count")
)

# Convert date back to datetime for merging
daily_rides["date"] = pd.to_datetime(daily_rides["date"])

print("Daily rides dataset shape:", daily_rides.shape)
daily_rides.head()

Daily rides dataset shape: (364, 2)


,date,daily_ride_count
0,2022-01-01,20428
1,2022-01-02,43009
2,2022-01-03,33189
3,2022-01-04,36842
4,2022-01-05,34230


In [9]:
# ==================================================
# FIX: KEEP ONLY 2022 ROWS, THEN RE-AGGREGATE DAILY
# ==================================================

# Ensure datetime
citibike_df["started_at"] = pd.to_datetime(citibike_df["started_at"], errors="coerce")

# Filter to 2022 only
citibike_2022 = citibike_df[
    (citibike_df["started_at"] >= "2022-01-01") &
    (citibike_df["started_at"] <  "2023-01-01")
].copy()

# Daily aggregation
citibike_2022["date"] = citibike_2022["started_at"].dt.floor("D")

daily_rides_2022 = (
    citibike_2022
    .groupby("date")
    .size()
    .reset_index(name="daily_ride_count")
    .sort_values("date")
)

print("Filtered rows (2022 only):", len(citibike_2022))
print("Daily rides dataset shape (2022):", daily_rides_2022.shape)
print("Date range:", daily_rides_2022["date"].min(), "to", daily_rides_2022["date"].max())

daily_rides_2022.head()

Filtered rows (2022 only): 29806700
Daily rides dataset shape (2022): (364, 2)
Date range: 2022-01-01 00:00:00 to 2022-12-30 00:00:00


,date,daily_ride_count
0,2022-01-01,20428
1,2022-01-02,43009
2,2022-01-03,33189
3,2022-01-04,36842
4,2022-01-05,34230


In [10]:
# ==================================================
# 6. MERGE DAILY RIDES WITH WEATHER DATA
# ==================================================

# Make sure weather date column is datetime
noaa_df["date"] = pd.to_datetime(noaa_df["date"])

# Merge on date
final_df = pd.merge(
    daily_rides,
    noaa_df,
    on="date",
    how="inner"   # keep only dates that exist in both
)

print("Merged dataset shape:", final_df.shape)
final_df.head()

Merged dataset shape: (364, 6)


,date,daily_ride_count,PRCP,TAVG,TMAX,TMIN
0,2022-01-01,20428,19.3,11.6,13.9,10.0
1,2022-01-02,43009,1.0,11.4,15.6,3.9
2,2022-01-03,33189,0.0,1.4,3.9,-4.3
3,2022-01-04,36842,0.0,-2.7,2.2,-6.0
4,2022-01-05,34230,6.1,3.2,8.9,0.0


In [11]:
# ==================================================
# 9. EXPORT MERGED DATASET
# ==================================================

final_df.to_csv("citibike_2022_daily_with_weather.csv", index=False)

print("Exported: citibike_2022_daily_with_weather.csv")

Exported: citibike_2022_daily_with_weather.csv


In [12]:
# ===============================
# 2. EXTRACT MAIN 2022 ZIP FILE
# ===============================

main_zip_path = "data/2022-citibike-tripdata.zip"
extract_path = "data/"

with zipfile.ZipFile(main_zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

print("Main 2022 zip extracted.")

Main 2022 zip extracted.


In [ ]:
# ==========================================
# 3. EXTRACT ALL MONTHLY ZIP FILES (12)
# ==========================================

monthly_folder = os.path.join("data", "2022-citibike-tripdata")

zip_files = [
    os.path.join(monthly_folder, f)
    for f in os.listdir(monthly_folder)
    if f.lower().endswith(".zip")
]

print("Monthly zip files found:", len(zip_files))

for z in zip_files:
    with zipfile.ZipFile(z, "r") as zip_ref:
        zip_ref.extractall(monthly_folder)

print("All monthly zip files extracted.")

Monthly zip files found: 12


In [7]:
# ==================================================
# 4. COMBINE ALL CSV FILES INTO ONE DATAFRAME
# Uses:
# 1) List comprehension to build list of CSV file paths
# 2) Generator expression inside pd.concat() for memory efficiency
# ==================================================

import os
import pandas as pd

# IMPORTANT:
# Your CSVs are inside this folder (based on your screenshot):
# data/2022-citibike-tripdata/
monthly_folder = os.path.join("data", "2022-citibike-tripdata")

# 1) List comprehension: builds a Python list of full file paths for every .csv in the folder
csv_files = [
    os.path.join(monthly_folder, f)
    for f in os.listdir(monthly_folder)
    if f.lower().endswith(".csv")
]

print("CSV files found:", len(csv_files))
print("First 3 files:", csv_files[:3])

if len(csv_files) == 0:
    raise ValueError(
        f"No CSV files found in '{monthly_folder}'. "
        f"Check the folder path and make sure the CitiBike CSV files exist there."
    )

# 2) Generator expression inside pd.concat(): reads each CSV one-by-one without building a giant list of DataFrames
citibike_df = pd.concat(
    (pd.read_csv(file, low_memory=False) for file in csv_files),
    ignore_index=True
)

print("All CSV files combined successfully.")
print("Dataset shape:", citibike_df.shape)

citibike_df.head()

CSV files found: 36
First 3 files: ['data/2022-citibike-tripdata/202208-citibike-tripdata_3.csv', 'data/2022-citibike-tripdata/202207-citibike-tripdata_2.csv', 'data/2022-citibike-tripdata/202207-citibike-tripdata_3.csv']
All CSV files combined successfully.
Dataset shape: (29838806, 13)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,9D0DC440CB40CF8E,electric_bike,2022-08-27 13:56:47.728,2022-08-27 14:02:56.651,Flatbush Ave & Ocean Ave,3704.04,3 St & Prospect Park West,3865.05,40.663657,-73.963014,40.668132,-73.973638,casual
1,2214991DFBE5C4D7,electric_bike,2022-08-20 10:37:02.756,2022-08-20 10:45:56.631,Forsyth St\t& Grand St,5382.07,E 11 St & 1 Ave,5746.14,40.717798,-73.993161,40.729538,-73.984267,casual
2,20C5D469563B6337,classic_bike,2022-08-31 18:55:03.051,2022-08-31 19:03:37.344,Perry St & Bleecker St,5922.07,Grand St & Greene St,5500.02,40.735354,-74.004831,40.721700,-74.002381,member
3,3E8791885BC189D1,classic_bike,2022-08-02 08:05:00.250,2022-08-02 08:16:52.063,FDR Drive & E 35 St,6230.04,Grand Army Plaza & Central Park S,6839.10,40.744219,-73.971212,40.764397,-73.973715,member
4,8DBCBF98885106CB,electric_bike,2022-08-25 15:44:48.386,2022-08-25 15:55:39.691,E 40 St & 5 Ave,6474.11,Ave A & E 14 St,5779.11,40.752052,-73.982115,40.730311,-73.980472,member
